In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
from tqdm.auto import tqdm
tqdm.pandas()

/home/dbsm-user/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [12]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query    
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content, 'lxml')
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content, 'lxml')
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

In [13]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    """ # umfang
    umfang = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        umfang = umfang[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        umfang = "unkown" 
        
    meta_dict = {"idn":idn,
                 "umfang":umfang}"""
    
    return idn

In [14]:
def make_list(row):
    if len(str(row.ISBN)) > 0:
        return list(map(parse_record, dnb_sru(f'NUM={row.ISBN}')))
    else:
        return list(map(parse_record, dnb_sru(f'WOE all {row.Titel} {row.Verlag} {row.Datum}')))


https://services.dnb.de/sru/dnb?version=1.1&operation=searchRetrieve&recordSchema=MARC21-xml&query=woe=Besuch bei Jitka Junge Welt, Berlin 1978

In [19]:
df = pd.read_csv('popups.csv', sep=';')

df['dnb-idns'] = df.progress_apply(make_list, axis=1)
df.to_csv('2021-04-20-popups.csv', sep=';', index=None)

100%|██████████| 4276/4276 [12:11<00:00,  5.84it/s]


In [9]:
list(map(parse_record, dnb_sru(f'WOE all Besuch bei Jitka Junge Welt, Berlin 1978')))

['790073773']

In [20]:
df

,ISBN,EAN,ASIN,Eigenschaft,Class,AX,BX,Autor,Titel,Seiten,...,Orginalsprache,Reihe,Thema,Buchart,Ersterscheinung,Standort,Auflage,Inhalt,Buchnummer,dnb-idns
0,NaN,978.0,NaN,8401089,Original,NaN,"Arndt, Paul","Arndt, Paul (Einl.); Schanz, Frida u. Pauline;...",Das sprechende Bilderbuch,17.0,...,NaN,NaN,"Hahn, Henne,Esel, Lamm, Vogel, Nest, Kuh, Kind...",NaN,3,NaN,4.,In dem Buch werden 8 Bauernhoftiere mit je ein...,4244.0,[]
1,NaN,978.0,NaN,8455201,Original,NaN,-,"Stanovsky, V. (Text); Kibasta, V. (Ill.)",Von der kleinen Mette - Janies Abenteuer bei...,44.0,...,NaN,Märchen aus aller Welt,"Dorf, Prinz, Schäferin,Pferde, König, Kutsche,...",NaN,3,NaN,NaN,NaN,4243.0,[]
2,NaN,978.0,NaN,206210,Übersetzung USA,NaN,N.N.,"Disney, Walt",Donald Duck in der Geisterstad,10.0,...,NaN,Zwei-Seiten-Panorama,"Donald Duck, Dagobert, Neffen, Büro,Auto, West...",NaN,2,6 - 2,NaN,"Donald Duck, seine 3 Neffen und Dagobert fahre...",4242.0,[]
3,3407803702,978.0,3407803702,9153,NaN,NaN,-,"Pacovská, Kveta(Ill.); Schwarz Annelies (Text)",Das Tier mit den Funkelaugen,32.0,...,NaN,NaN,"Tier, Fantasie, Kind Rikki, BettTraum",NaN,2,3 - 4,NaN,"Rikki wünscht sich ein Tier, dass fliegen, Sch...",4241.0,[900649100]
4,NaN,978.0,NaN,66689,NaN,NaN,N:N.,"Eichelberger, Ursula; Zucker, Gertrud",Besuch bei Jitka,10.0,...,NaN,Kofferbücher,"Prag, Löwe, Karlsbrücke,Hochzeit, Kinder, Uhr,...",NaN,2,6 - 1,1.,"Jitka kommt aus Prag und erzählt, was in ihrer...",4240.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4271,NaN,978.0,NaN,524292,Nachdruck,NaN,N.N.,N.N.,Der große Brand in Hamburg 1842,6.0,...,NaN,NaN,Amerikanische Kinderreime,Teleorama,2,K - 8,NaN,"Das ""Buch"" besteht aus sechs durch weiße Seite...",3.0,[]
4272,NaN,978.0,NaN,12,Faksimile,NaN,"Flach, Martin","Flach, Martin; Voulliéme",Losbuch,8.0,...,NaN,NaN,NaN,Scherzhaftes Wahrsagebuch,1,K - 1,NaN,Scherzhaftes Wahrsagebuch mit einem D rachenko...,2.0,[]
4273,NaN,978.0,NaN,130,Original,NaN,"Gould, Chester","Gould, Chester","The ""Pop-up"" Dick Tracy ""Capture of Boris Arson""",18.0,...,NaN,NaN,Krimi,NaN,2,1 - 7,NaN,Eine wilde Vverfolgungsjagd nach dem entflohen...,74.0,[]
4274,NaN,NaN,NaN,2052,Faksimile,NaN,"Apianus, Petrus","Apianus, Petrus,Bienewitz, Peter; Wattehberg, ...",Astronomicum Caesareum,124.0,...,NaN,NaN,Astronomie,Fachbuch,2,K - 0,NaN,Die Faksimile-Ausgabe wurde 1967 in Leipzig ...,1.0,[]
